## MNIST con EMNIST features

En este notebook se entrena una red que utiliza los features aprendidos a partir de la clasificación de EMNIST para clasificar datos de la base de datos de MNIST. Para ello se hace lo mismo de siempre, se divide la red en features layers y classification layes, se copian los pesos de la parte de feature layers y se entrenan los pesos de la parte de clasificacion utilizando la nueva base de datos. Se obtuvo con 15 epocas un accuracy de 99.25%.

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
import imageio
from scipy.misc import imread, imsave, imresize
import PIL
from PIL import Image
from skimage import transform,io
from resizeimage import resizeimage
import scipy.io as sio
import scipy.misc
#import ImageOps
from os import listdir
from os.path import isfile, join
from skimage.transform import rescale, resize as rs, downscale_local_mean

/home/cuchuflito/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
model=load_model('modelo_EMNIST_ok.h5')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train.shape)

In [47]:
#leido de csv con pd
#Y_train = df_XY_train['label'].values
#X_train = df_XY_train.drop('label', axis=1).values
#X_test  = df_X_test.values

In [6]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1) #tensorflow channels_last
num_classes = 10

In [7]:
X_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1).astype('float32')/255
Y_train = keras.utils.to_categorical(y_train, num_classes)
X_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1).astype('float32')/255
Y_test = keras.utils.to_categorical(y_test, num_classes)

In [18]:
feature_layers = [
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = (28,28,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Flatten(),
]

In [19]:
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
]
for l in feature_layers:
    l.trainable = False

In [20]:
newmodel=Sequential(feature_layers+classification_layers)

In [21]:
#model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
weights= [layer.get_weights() for layer in model.layers]
for i in np.arange(0,len(feature_layers),1):
        newmodel.layers[i].set_weights(weights[i])

In [54]:
newmodel.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
newmodel.fit(X_train, Y_train, batch_size=128, epochs=15, verbose=1,validation_data=(X_test,Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 65s 1ms/step - loss: 0.1377 - acc: 0.9581 - val_loss: 0.0351 - val_acc: 0.9882
Epoch 2/15
60000/60000 [==============================] - 60s 996us/step - loss: 0.0592 - acc: 0.9822 - val_loss: 0.0279 - val_acc: 0.9908
Epoch 3/15
60000/60000 [==============================] - 60s 998us/step - loss: 0.0458 - acc: 0.9863 - val_loss: 0.0278 - val_acc: 0.9907
Epoch 4/15
60000/60000 [==============================] - 58s 964us/step - loss: 0.0375 - acc: 0.9888 - val_loss: 0.0279 - val_acc: 0.9909
Epoch 5/15
60000/60000 [==============================] - 55s 909us/step - loss: 0.0330 - acc: 0.9894 - val_loss: 0.0254 - val_acc: 0.9919
Epoch 6/15
60000/60000 [==============================] - 54s 905us/step - loss: 0.0282 - acc: 0.9913 - val_loss: 0.0264 - val_acc: 0.9915
Epoch 7/15
60000/60000 [==============================] - 55s 912us/step - loss: 0.0251 - acc: 0.9922 - val_loss: 0.02

In [15]:
newmodel.save('MNIST_with_EMNIST_features.h5')


In [5]:
for layers in model.layers:
	layers.trainable=False
model.layers[-1].trainable=True
model.layers[-3].trainable=True

In [14]:
newmodel.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
hist=newmodel.fit(X_train, Y_train, batch_size=128*4, epochs=20, verbose=1,validation_data=(X_test,Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 69s 1ms/step - loss: 0.1312 - acc: 0.9607 - val_loss: 0.0506 - val_acc: 0.9838
Epoch 2/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0726 - acc: 0.9784 - val_loss: 0.0411 - val_acc: 0.9866
Epoch 3/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0566 - acc: 0.9832 - val_loss: 0.0307 - val_acc: 0.9892
Epoch 4/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0482 - acc: 0.9854 - val_loss: 0.0301 - val_acc: 0.9903
Epoch 5/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0421 - acc: 0.9877 - val_loss: 0.0267 - val_acc: 0.9908
Epoch 6/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0368 - acc: 0.9888 - val_loss: 0.0268 - val_acc: 0.9906
Epoch 7/20
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0346 - acc: 0.9893 - val_loss: 0.0294 - val_acc

In [15]:
import json
with open('MNIST_with_EMNIST_features.json', 'w') as f:
    json.dump(hist.history, f)


In [22]:
newmodel.layers[-1].trainable=True
newmodel.layers[-3].trainable=False
newmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [23]:
newmodel.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
hist=newmodel.fit(X_train, Y_train, batch_size=128*4, epochs=20, verbose=1,validation_data=(X_test,Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 63s 1ms/step - loss: 2.0154 - acc: 0.3141 - val_loss: 1.5295 - val_acc: 0.7560
Epoch 2/20
60000/60000 [==============================] - 65s 1ms/step - loss: 1.5050 - acc: 0.5569 - val_loss: 1.1457 - val_acc: 0.8407
Epoch 3/20
60000/60000 [==============================] - 65s 1ms/step - loss: 1.2768 - acc: 0.6256 - val_loss: 0.9362 - val_acc: 0.8614
Epoch 4/20
60000/60000 [==============================] - 64s 1ms/step - loss: 1.1552 - acc: 0.6549 - val_loss: 0.8080 - val_acc: 0.8700
Epoch 5/20
60000/60000 [==============================] - 64s 1ms/step - loss: 1.0819 - acc: 0.6710 - val_loss: 0.7233 - val_acc: 0.8768
Epoch 6/20
60000/60000 [==============================] - 65s 1ms/step - loss: 1.0395 - acc: 0.6745 - val_loss: 0.6645 - val_acc: 0.8824
Epoch 7/20
60000/60000 [==============================] - 65s 1ms/step - loss: 1.0115 - acc: 0.6790 - val_loss: 0.6234 - val_acc

In [24]:
with open('MNIST_with_EMNIST_features_1.json', 'w') as f:
    json.dump(hist.history, f)
